# This is a code used to detect faces in videos and extract gaze vector.


## First part of the code is setting up the enviroment (if you are new to coding this wont be legible to you)


In [ ]:
!pip install protobuf==3.20.1
!pip install --default-timeout=100 tensorflow==2.8.0
import tensorflow    # it is used to contruct neural networks
print(tensorflow.__version__)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-api-core 2.19.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-aiplatform 1.73.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-cloud-bigquery-connection 1.16.1 requir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 44.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

After restarting the runtime, verify that your version is 2.8.0

In [ ]:
import tensorflow
print(tensorflow.__version__)

from google.colab import drive
drive.mount('/content/drive')

!pip install --default-timeout=100 visvis

2.8.0
Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 58.2 MB/s eta 0:00:00


### Uploading the models of Gaze360 and Yolo

In [ ]:
! git clone https://github.com/erkil1452/gaze360
! wget http://gaze360.csail.mit.edu/files/gaze360_model.pth.tar

import random
from PIL import Image
import math
import numpy as np
import os
import imageio
import time
import datetime

import sys
import torch
import torchvision.transforms as transforms
import argparse
import cv2
import json

# This function will be useful for graphical representation of gaze
def spherical2cartesial(x):
    output = torch.zeros(x.size(0),3)
    output[:,2] = -torch.cos(x[:,1])*torch.cos(x[:,0])
    output[:,0] = torch.cos(x[:,1])*torch.sin(x[:,0])
    output[:,1] = torch.sin(x[:,1])
    return output

# This is the path you will have to change according to the postion of files
sys.path.append('/content/drive/MyDrive/PW_2024/PW_2024/HeadPoseEstimation-WHENet/')
sys.path.append('/content/gaze360/code/')
from model import GazeLSTM
model = GazeLSTM() #name of the model; short memory. RGB images are put inside.
model = torch.nn.DataParallel(model).cuda()
model.cuda()
checkpoint = torch.load('/content/gaze360_model.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print(sys.path)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 106MB/s]
<ipython-input-4-b8a243d3f1b8>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded 

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmprh05pt0z', '/content/drive/MyDrive/PW_2024/PW_2024/HeadPoseEstimation-WHENet/', '/content/gaze360/code/']


Pass the arguments required by Retina face detector

In [ ]:
!pip install retina-face
!pip install opencv-python-headless
from retinaface import RetinaFace

# Loading pre-trained model
image_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


## This is the main code

It will look for video and tempos folder within your colab content and return a feature vector as txt in the desired location, additionaly if the writing_mode = 1 then it will depict in a new video the result of gaze detection.

In [ ]:
import os
import cv2
import imageio
import numpy as np
import random
import torch
from PIL import Image
from torchvision import transforms

# Configuration
writing_mode = 0
mydirectory = '/content'#/drive/MyDrive/PW_2024/PW_2024/Gabriele_Laura/'
path_to_write = mydirectory#'/content/drive/MyDrive/'
f4 = open(os.path.join(path_to_write, 'angle_gaze360.txt'), 'w')

all_files = os.listdir(mydirectory)

# Color for the arrow
get_color = lambda t_id: (255, 0, 0) if t_id == 0 else (0, 255, 0) if t_id == 1 else (0, 0, 255)

# Open video and metadata
video_name = next((s for s in all_files if 'video' in s and 'yolo' not in s and 'mtcnn' not in s and 'labels' not in s and 'prueba' not in s), None)
v_reader = imageio.get_reader(os.path.join(mydirectory, video_name))
fps = v_reader.get_meta_data()['fps']

# Prepare video writer and gaze log
if writing_mode == 1:
    outfile = os.path.join(path_to_write, 'video_gaze360.mp4')
    out = imageio.get_writer(outfile, fps=fps)

# Read timing information
time_cuts_name = next((s for s in all_files if 'tempos' in s), None)
time_cuts = open(os.path.join(mydirectory, time_cuts_name)).read().splitlines()
video_length = len(time_cuts)
print(f'Video length in frames: {video_length}')
print(f'Start time: {time_cuts[0]}')
print(f'End time: {time_cuts[-1]}')

# Scaling is important for Gaze360
WIDTH, HEIGHT = 960, 720
W = 5 # Window size
scale_width = WIDTH / v_reader.get_meta_data()['size'][0]
scale_height = HEIGHT / v_reader.get_meta_data()['size'][1]

# Process video frames Retina
tracking_id = {}
image_stack = []
cnt = 0
for frame_idx, image in enumerate(v_reader):
    if frame_idx % 5 != 0: continue
    print(frame_idx, video_length)
    # Detect faces and landmarks
    image = cv2.resize(image,(WIDTH,HEIGHT))
    faces = RetinaFace.detect_faces(image)
    if not faces:
        continue

    # Update tracking information
    frame_results = {}
    for face_id, face_info in faces.items():
        bbox = face_info["facial_area"]
        right_eye = face_info["landmarks"]["right_eye"]
        left_eye = face_info["landmarks"]["left_eye"]
        frame_results[int(face_id[-1])] = (bbox, left_eye, right_eye)
    tracking_id[cnt] = frame_results
    cnt += 1

    image_stack.append(image)

# Process video frames Gaze
for frame_idx in range(0, len(image_stack)):
    print(frame_idx, video_length)
    if writing_mode == 1:
        image_copy = image_stack[frame_idx].copy().astype(float)

    if frame_idx in tracking_id.keys():
        for face_idx in tracking_id[frame_idx].keys():
            # Prepare input tensor for gaze prediction
            input_image = torch.zeros(7, 3, 224, 224)
            cnt = 0
            for j in range(frame_idx - 3 * W, frame_idx + 4 * W, W):
                if j in tracking_id and face_idx in tracking_id[j]:
                    crop_img = Image.fromarray(image_stack[j],'RGB')
                    bbox, left_eye, right_eye = tracking_id[j][face_idx]
                else:
                    crop_img = Image.fromarray(image_stack[frame_idx],'RGB')
                    bbox, left_eye, right_eye = tracking_id[frame_idx][face_idx]
                crop_img = crop_img.crop((bbox[0], bbox[1], bbox[2], bbox[3]))
                input_image[cnt] = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    image_normalize
                ])(crop_img)
                cnt = cnt+1

            # Predict gaze
            output_gaze, output_confidence = model(input_image.view(1, 7, 3, 224, 224).cuda())
            gaze = spherical2cartesial(output_gaze).detach().cpu().numpy().flatten()
            # When working with full size images
            eyes = np.array([(left_eye[0] + right_eye[0])/2.0, (left_eye[1] + right_eye[1])/2.0])
            # eyes = np.array([scale_width * eyes[0], scale_height * eyes[1]])
            bbox = np.array(bbox) # * np.array([scale_width, scale_height] * 2)

            if writing_mode == 1:
                # Draw gaze
                image_copy = cv2.rectangle(image_copy, tuple(bbox[:2]), tuple(bbox[2:]), get_color(face_idx))
                end_point = (int(eyes[0] - gaze[0] * 100), int(eyes[1] - gaze[1] * 100))
                image_copy = cv2.arrowedLine(image_copy.astype(np.uint8), tuple(eyes.astype(int)), end_point, get_color(face_idx), 6)

            # Write gaze data
            # Tensor to Numpy
            confidence = output_confidence.detach().cpu().numpy().flatten()
            gaze = output_gaze.detach().cpu().numpy().flatten()

            bbox_str = f"{bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}"
            line_to_write = f"{frame_idx} {confidence[0]:.3f} {confidence[1]:.3f} {gaze[0]:.3f} {gaze[1]:.3f} {bbox_str} {left_eye[0]:.3f} {left_eye[1]:.3f} {right_eye[0]:.3f} {right_eye[1]:.3f}\n"
            print(line_to_write)
            f4.write(line_to_write)

        if writing_mode == 1:
            out.append_data(image_copy.astype(np.uint8))

# Cleanup
if writing_mode == 1:
    out.close()
f4.close()


Video length in frames: 547
Start time: 1732532474.9844575
End time: 1732532531.9817548
0 547
5 547
10 547
15 547
20 547
25 547
30 547
35 547
40 547
45 547
50 547
55 547
60 547
65 547
70 547
75 547
80 547
85 547
90 547
95 547
100 547
105 547
110 547
115 547
120 547
125 547
130 547
135 547
140 547
145 547
150 547
155 547
160 547
165 547
170 547
175 547
180 547
185 547
190 547
195 547
200 547
205 547
210 547
215 547
220 547
225 547
230 547
235 547
240 547
245 547
250 547
255 547
260 547
265 547
270 547
275 547
280 547
285 547
290 547
295 547
300 547
305 547
310 547
315 547
320 547
325 547
330 547
335 547
340 547
345 547
350 547
355 547
360 547
365 547
370 547
375 547
380 547
385 547
390 547
395 547
400 547
405 547
410 547
415 547
420 547
425 547
430 547
435 547
440 547
445 547
450 547
455 547
460 547
465 547
470 547
475 547
480 547
485 547
490 547
495 547
500 547
505 547
510 547
515 547
520 547
525 547
530 547
535 547
540 547
545 547
0 547
0 0.301 0.301 0.401 -0.360 658 412 687 458 668.2